In [1]:
import numpy as np

In [2]:
import clease
from clease.settings import Concentration
import numpy as np
from clease.settings import CEBulk
from clease.structgen import NewStructures
from ase.db import connect
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter
import json, os 


In [3]:
def closest_composition(comp, num_atoms):
    # Calculate the total fraction
    total_fraction = sum(comp.values())
    
    # Calculate the number of atoms for each element
    atoms = {element: fraction / total_fraction * num_atoms for element, fraction in comp.items()}
    
    # Round up non-integer atom counts and adjust the first element to balance
    elements = list(atoms.keys())
    for i in range(1, len(elements)):
        if atoms[elements[i]] % 1 != 0:
            atoms[elements[i]] = int(atoms[elements[i]]) + 1
            #num_atoms += 1

    atoms[elements[0]] = num_atoms - sum(atoms[elements[i]] for i in range(1, len(elements)))
    
    # Calculate the actual fractions
    actual_fractions = {element: num / num_atoms for element, num in atoms.items()}
    
    return atoms, actual_fractions

def create_random_supercells(composition,alat,supercell_size,db_name,num_structures):
    A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
    b_eq = composition.values()
    conc = Concentration(basis_elements=[list(composition.keys())],
                         A_eq = A_eq,
                         b_eq = b_eq)
    settings = CEBulk(crystalstructure='bcc',
                      a=alat,
                      size=[supercell_size,supercell_size,supercell_size],
                      concentration=conc,
                      db_name=db_name,
                      max_cluster_dia=[6.0, 4.5, 4.5])
    


    ns = NewStructures(settings, generation_number=1,
                    struct_per_gen=num_structures)
    ns.generate_random_structures()

def connect_ase_db(db_name):
    from ase.db import connect
    atoms = connect(db_name)
    print(len(atoms))
    for idx in range(len(atoms)):
        curr_struct = atoms.get(id=idx+1).toatoms()
        print(curr_struct)
    return atoms

        
def ase_db_to_pymatgen(db_name,output_file=None):

    data = {}
    Adaptor = AseAtomsAdaptor()
    atoms = connect(db_name) 

    for idx in range(2,len(atoms)+1):
        
        curr_struct = atoms.get(id=idx).toatoms()
        
        curr_pymatgen = Adaptor.get_structure(curr_struct)
        
        
        trans = PerturbStructureTransformation(distance=0.1, min_distance=0.01)
        distorted_struct = trans.apply_transformation(curr_pymatgen)
        
        data = {f'{idx}':distorted_struct.as_dict()}
        
        if output_file is not None:
            with open(output_file, 'w') as f:
                json.dump(data, f,)
        else:
            return data
        

In [4]:
comp = {'V' : 0.92 , 'Cr' : 0.04, 'Ti' : 0.04}
num_atoms = 64
atoms, actual_fractions = closest_composition(comp, num_atoms)

print("Number of atoms for each element:", atoms)
print("Actual fractions for each element:", actual_fractions)

Number of atoms for each element: {'V': 58, 'Cr': 3, 'Ti': 3}
Actual fractions for each element: {'V': 0.90625, 'Cr': 0.046875, 'Ti': 0.046875}


In [5]:
create_random_supercells(composition=comp,
                         alat = 3.0,
                         supercell_size=4,
                         db_name='../CLEASE_DBs/v4cr4ti.db',
                         num_structures=30)

InvalidConstraintError: b_eq has to be a 1D vector!

In [6]:
from clease.settings import Concentration

A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.90625,0.046875,0.046875]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [7]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="../CLEASE_DBs/V4Cr4Ti.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [8]:
from clease.structgen import NewStructures


ns = NewStructures(settings, generation_number=1,
                   struct_per_gen=10)
ns.generate_random_structures()


In [11]:
A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.609375,0.296875,0.09375]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [12]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="../CLEASE_DBs/V30Cr9_5Ti.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [13]:

ns = NewStructures(settings, generation_number=1,
                   struct_per_gen=10)
ns.generate_random_structures()

In [14]:
A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.609375,0.09375,0.296875]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [15]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="../CLEASE_DBs/V9_5Cr30Ti.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [16]:

ns = NewStructures(settings, generation_number=1, # was 3 if same db
                   struct_per_gen=10)
ns.generate_random_structures()

In [17]:
A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
b_eq = [0.765625,0.125,0.109375]
conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

In [18]:
from clease.settings import CEBulk
settings = CEBulk(crystalstructure='bcc',
                  a=3.0,
                  size=[4,4,4],
                  concentration=conc,
                  db_name="../CLEASE_DBs/V12_6Cr11_1Ti.db",
                  max_cluster_dia=[6.0, 4.5, 4.5])

In [19]:
ns = NewStructures(settings, generation_number=1, # was 3 if same db
                   struct_per_gen=10)
ns.generate_random_structures()

In [6]:
x_cr = 12
x_ti = 6
# for 15 picks, choose from 1 to 19 for x_cr and 1 to 13 for x_ti 
# set the seed to 42
X_Cr = []
X_Ti = []
np.random.seed(42)
for i in range(10):
    x_cr = np.random.randint(1,20)
    x_ti = np.random.randint(1,14)

    # check if the composition is already in the list
    if [x_cr,x_ti] in X_Cr and X_Ti:
        # if it is, generate a new composition
        while [x_cr,x_ti] in X_Cr and X_Ti:
            x_cr = np.random.randint(1,20)
            x_ti = np.random.randint(1,14)

    X_Cr.append(x_cr)
    X_Ti.append(x_ti)

    print("X_cr = ",x_cr, "X_ti = ",x_ti)

    A_eq = [[1.0, 0.0, 0.0],[0.0, 1.0, 0.0],[0.0, 0.0, 1.0]]
    b_eq = [1-x_cr/64-x_ti/64,x_cr/64,x_ti/64]
    conc = Concentration(basis_elements=[["V","Cr","Ti"]], A_eq=A_eq, b_eq=b_eq)

    from clease.settings import CEBulk

    settings = CEBulk(crystalstructure='bcc',
                    a=3.0,
                    size=[4,4,4],
                    concentration=conc,
                    db_name=f"../CLEASE_DBs/V{x_cr}Cr{x_ti}Ti.db",
                    max_cluster_dia=[6.0, 4.5, 4.5])


    ns = NewStructures(settings, generation_number=1, # was 3 if same db
                    struct_per_gen=3)
    ns.generate_random_structures()

X_cr =  7 X_ti =  4
X_cr =  16 X_ti =  3
X_cr =  9 X_ti =  10
X_cr =  2 X_ti =  7
X_cr =  15 X_ti =  5
X_cr =  16 X_ti =  10
X_cr =  18 X_ti =  5
X_cr =  17 X_ti =  13
X_cr =  2 X_ti =  3
X_cr =  15 X_ti =  10


In [12]:
from ase.db import connect
atoms = connect('V_Cr_Ti_random_supercells_Nov_29_2023.db')
print(len(atoms))
for idx in range(len(atoms)):
    curr_struct = atoms.get(id=idx+1).toatoms()
    print(curr_struct)

31
Atoms(symbols='V', pbc=True, cell=[[-1.5, 1.5, 1.5], [1.5, -1.5, 1.5], [1.5, 1.5, -1.5]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr3Ti3V58', pbc=Tr

In [13]:
from ase.io import write
from pymatgen.io.ase import AseAtomsAdaptor
from pymatgen.transformations.standard_transformations import PerturbStructureTransformation
from pymatgen.io.cif import CifWriter

Adaptor = AseAtomsAdaptor()

for idx in range(2,len(atoms)+1):
    
    curr_struct = atoms.get(id=idx).toatoms()
    
    curr_pymatgen = Adaptor.get_structure(curr_struct)
    
    
    trans = PerturbStructureTransformation(distance=0.1, min_distance=0.01)
    distorted_struct = trans.apply_transformation(curr_pymatgen)
    
    writer = CifWriter(distorted_struct)
    writer.write_file("VCrTi_Supercells_Nov_29_2023/Struct_num"+str(idx)+".cif")

In [14]:
print(len(atoms))

31


In [7]:
from ase.db import connect
from pymatgen.io.ase import AseAtomsAdaptor
dbs = ['../CLEASE_DBs/V4Cr4Ti.db',
       '../CLEASE_DBs/V30Cr9_5Ti.db',
       '../CLEASE_DBs/V9_5Cr30Ti.db',
       '../CLEASE_DBs/V12_6Cr11_1Ti.db'
       ]

# create a list of all the *.db files in db_path
db_path = '../CLEASE_DBs/VCrTi'
db_files = [os.path.join(db_path, f) for f in os.listdir(db_path) if f.endswith('.db')]


structures = []
k = 0
for db in db_files:
    atoms = connect(db)
    for idx in range(len(atoms)):
        curr_struct = atoms.get(id=idx+1).toatoms()
        print(curr_struct)

Atoms(symbols='V', pbc=True, cell=[[-1.5, 1.5, 1.5], [1.5, -1.5, 1.5], [1.5, 1.5, -1.5]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', pbc=True, cell=[[-6.0, 6.0, 6.0], [6.0, -6.0, 6.0], [6.0, 6.0, -6.0]], tags=...)
Atoms(symbols='Cr19Ti6V39', 

In [ ]:
# for making jobs from mcmc 
#db_directory = '/Users/myless/Packages/structure_maker/VCrTiWZr_DBs'
mcmc_directory = '../Visualization/Job_Structures'
vasp_job_path = '../Archived_Vasp_Jobs/VCrTiWZr_PostMCMC'
#num_structures = 10
#supercell_size = 4
#num_atoms = supercell_size**3
structures = []
k = 0 
if not os.path.exists(vasp_job_path):
    os.makedirs(vasp_job_path)

# create a json that stores the structure name and the job number 
structure_dict = {}
# load in the structures from the mcmc directory 
# each one will be a cif file, that ends with '_final.cif'
# get a list for all .cif files
cif_files = [f for f in os.listdir(mcmc_directory) if f.endswith('.cif')]
for i, cif_file in enumerate(cif_files):
    structure_dict[str(k)] = cif_file
    supercell = Structure.from_file(os.path.join(mcmc_directory,cif_file))
    #new_atoms, new_atom_dict = closest_composition(composition, num_atoms)
    #print(new_atom_dict)
    #db_name = '_'.join([f'{key}{value}' for key, value in new_atom_dict.items()]) + '.db'
    #generate_supercells = create_random_supercells(new_atom_dict, 3.01, supercell_size, os.path.join(db_directory,db_name), num_structures)
    #generated_structures = ase_db_to_pymatgen(os.path.join(db_directory,db_name))
    job_path = os.path.join(vasp_job_path, f'job_{k}')
    os.makedirs(job_path, exist_ok=True)
    if isinstance(supercell, dict):
        supercell = Structure.from_dict(supercell)
        supercell = supercell.get_sorted_structure()
    make_vasp_job(supercell.get_sorted_structure(), job_path, kpoints_params=(3,3,3), incar_params=None)
    make_slurm_file(job_path, k, num_gpus=1,omp_threads=10)
    print(f'Job {k} made')
    k += 1 
    
# dump the json to the same directory as the structures
with open(os.path.join(vasp_job_path,'structure_dict.json'), 'w') as f:
    json.dump(structure_dict, f)